### Exercise 2 : Create a program to ask a model to list all European countries, with their capital city. Create Pydantic classes torepresent the structure of the required response (one class representing a City and one the collection of cities).

In [1]:
import os
import ast
from dotenv import load_dotenv
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from pydantic import BaseModel, Field
from openai import pydantic_function_tool
from langsmith import traceable
load_dotenv()

# read in env variables
api_key = os.getenv("GEMINI_API_KEY")
api_base = os.getenv("GEMINI_API_BASE")
model = os.getenv("GEMINI_API_MODEL")

In [2]:
client = wrap_openai(OpenAI(
 api_key=api_key,
 base_url=api_base,
))

In [3]:
# Define the data model for the math reasoning steps
class City(BaseModel):
    name: str = Field(description="The name of the city")
    country: str = Field(description="The country where the city is located")

# Define the data model for the overall math reasoning
class EuroCities(BaseModel):
    steps: list[City]


In [4]:
response = client.chat.completions.parse(
    model=model,
    messages=[

        {
            "role": "user",
            "content": "Provide a list of all European Captial Cities."
        }
    ],
    response_format=EuroCities
)

In [5]:
solution = response.choices[0].message.parsed

 # Print the final answer
for step in solution.steps:
    print(f"Country: {step.country}")
    print(f"Capital: {step.name}\n")

Country: Albania
Capital: Tirana

Country: Andorra
Capital: Andorra la Vella

Country: Austria
Capital: Vienna

Country: Belarus
Capital: Minsk

Country: Belgium
Capital: Brussels

Country: Bosnia and Herzegovina
Capital: Sarajevo

Country: Bulgaria
Capital: Sofia

Country: Croatia
Capital: Zagreb

Country: Cyprus
Capital: Nicosia

Country: Czech Republic
Capital: Prague

Country: Denmark
Capital: Copenhagen

Country: Estonia
Capital: Tallinn

Country: Finland
Capital: Helsinki

Country: France
Capital: Paris

Country: Germany
Capital: Berlin

Country: Greece
Capital: Athens

Country: Hungary
Capital: Budapest

Country: Iceland
Capital: Reykjavik

Country: Ireland
Capital: Dublin

Country: Italy
Capital: Rome

Country: Kosovo
Capital: Pristina

Country: Latvia
Capital: Riga

Country: Liechtenstein
Capital: Vaduz

Country: Lithuania
Capital: Vilnius

Country: Luxembourg
Capital: Luxembourg City

Country: North Macedonia
Capital: Skopje

Country: Malta
Capital: Valletta

Country: Moldova

### In the following exercise you will create a tool to read the content of a file and summarise it.
a.
Start by creating a tool which returns the content of a filename found in the current working directory (usePydantic to define the tool). Then send a request to the model to summarise the content of a named file, alsopassing the tool definition you created. Check the response contains the tool name and the name of the file as anargument.
<br>
<br>
<br>
b.
Write a function that takes the tool name and arguments returned from the LM call, and execute the function(should return the content of a file).
<br>
<br>
<br>
c.
From Q4, add the file content to the message context (as an assistant message). To ensure best results makesure it is clear that the text is the text of the requested file (e.g. filename=”….”). Now call the model again with theappended context and observe the response.

In [7]:
class getFileContent(BaseModel):
    """Return  tool for getting file content."""
    filename: str = Field(description="the name of the file found in current working directory")
    

    def getFileContent(self):
        with open(self.filename, "r") as f:
            content = f.read()
        return content


tool = pydantic_function_tool(getFileContent)

print(tool)

{'type': 'function', 'function': {'name': 'getFileContent', 'strict': True, 'parameters': {'description': 'Return  tool for getting file content.', 'properties': {'filename': {'description': 'the name of the file found in current working directory', 'title': 'Filename', 'type': 'string'}}, 'required': ['filename'], 'title': 'getFileContent', 'type': 'object', 'additionalProperties': False}, 'description': 'Return  tool for getting file content.'}}


In [8]:
messages = [{"role": "user", "content": "content of file 'test.txt"}]
response = client.chat.completions.create(
    model="gemini-2.5-flash-lite",
    messages=messages,
    tools=tool
)

In [9]:
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)

getFileContent
{"filename":"test.txt"}


In [10]:
function_call = response.choices[0].message.tool_calls[0].function
function_call

Function(arguments='{"filename":"test.txt"}', name='getFileContent')

In [12]:
tool_lookup = {"content" : getFileContent}
#print(tool_lookup['content'])
args = function_call.arguments
print(args)
#res = getFileContent.getFileContent(**args)
#res = tool_lookup['content'](**args).getFileContent()
#print(res)

{"filename":"test.txt"}


In [13]:
args = ast.literal_eval(args)

In [14]:
res = getFileContent(**args).getFileContent()

In [15]:
@traceable(name="multi_turn_conversation")
def multi_turn_conversation():
    """Demonstrate multi-turn conversation with context preservation"""

    # Initialize conversation history with the system message
    conversation_history = [
        {
            "role": "system",
            "content": "You are a helpful tutor."
        }
    ]

    # now add the first user message to the history
    conversation_history.append(
        {"role": "user", "content": "content of file 'test.txt'"})

    # now call the model to get the assistant's response
    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1,
        tools=tool
    )
    function_call = response.choices[0].message.tool_calls[0].function
    tool_lookup = {"content" : getFileContent}
    #print(tool_lookup['content'])
    args = function_call.arguments
    args = ast.literal_eval(args)
    res = getFileContent(**args).getFileContent()
    res = 'This is the contents of the requested file test.txt: ' + res

    # add the response to the conversation history
    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": res
    })

    # now ask a follow-up question
    conversation_history.append(
        {"role": "user", "content": "summarize content of file 'test.txt'"})

    # now call the model to get the assistant's response
    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1,
        tools=tool
    )
    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })
    print(assistant_response)
    '''
    conversation_history.append({
        "role": "user",
        "content": "How do I add items to it?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1
    )

    # now add the assistant's response to the history
    assistant_response = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })

    # ask another follow-up question
    conversation_history.append({
        "role": "user",
        "content": "What's the difference between append() and extend()?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        temperature=0.1
    )

    assistant_response = response.choices[0].message.content
    print
    conversation_history.append({
        "role": "assistant",
        "content": assistant_response
    })
    '''

    return conversation_history



if __name__ == "__main__":
    print(multi_turn_conversation())

The file 'test.txt' contains a biography of Thomas Jeffrey Hanks, an American actor and filmmaker born on July 9, 1956. He is recognized for his versatile roles in both comedy and drama, and is considered a global film star and American cultural icon. Hanks has received numerous awards, including two Academy Awards, seven Emmy Awards, and four Golden Globe Awards. He gained initial fame through comedy films like *Splash* and *Big*, and later won two consecutive Best Actor Academy Awards for his roles in *Philadelphia* and *Forrest Gump*. He has a notable collaborative history with director Steven Spielberg, working on films such as *Saving Private Ryan* and miniseries like *Band of Brothers*.
[{'role': 'system', 'content': 'You are a helpful tutor.'}, {'role': 'user', 'content': "content of file 'test.txt'"}, {'role': 'assistant', 'content': 'This is the contents of the requested file test.txt: Thomas Jeffrey Hanks (born July 9, 1956) is an American actor and filmmaker. Known for both 